In [1]:
import networkx as nx
from networkx.algorithms import bipartite

import pandas as pd
import sys
sys.path.append("..")
from network_analysis.core_periphery_sbm import core_periphery as cp
from network_analysis.core_periphery_sbm import model_fit as mf
from network_analysis.load_datasets import get_updated_shxco_data
from network_analysis.generate_network_metrics import *
from network_analysis.create_networks import *
from network_analysis.read_write_networks import * 
from network_analysis.generate_coreness import *
from network_analysis.visualize_networks import *
members_df, books_df, borrow_events, events_df = get_updated_shxco_data(get_subscription=False)

### Load Data

In [2]:
all_events = events_df[events_df.item_uri.isna() == False].copy()


In [3]:
member_attrs = {'uri': 'member_id'}
book_attrs = {'uri': 'item_uri'}
edge_attrs = {'weight': 'counts'}
node_attrs = {}
should_process = False
write_to_file = True
sk_metrics = ['katz', 'louvain']
link_metrics = ['pagerank', 'hubs', 'auth']
is_projected = True
all_events_grouped = all_events.groupby(
    ['member_id', 'item_uri']).size().reset_index(name='counts')

projected_members_graph, projected_members_nodelist, projected_members_edgelist, projected_members, projected_books_graph, projected_books_nodelist, projected_books_edgelist, projected_books = check_reload_build_unipartite_graphs(
    all_events_grouped, all_events, member_attrs, book_attrs, edge_attrs, node_attrs, should_process, write_to_file, './data/borrow_events_unipartite_projected', sk_metrics, link_metrics, members_df, books_df, is_projected)


reloading saved graph: ./data/borrow_events_unipartite_projected


In [4]:
updated_projected_members_nodelist = get_core_scores(projected_members_graph)

Initializing hub-and-spoke model...
Done. Time taken: 259.13 seconds
Initializing layered model...
Done. Time taken: 382.05 seconds
Hub-and-Spoke Model: 47287.585643733735 Layered Model: 45446.15395615674
hubs and spokes global_hub_spoke
0    154
1    444
dtype: int64
layers global_layer
0     94
1    100
2    160
3    244
dtype: int64


In [5]:
mapping = dict(updated_projected_members_nodelist[['label', 'uri']].values)
H = nx.relabel_nodes(projected_members_graph, mapping)


In [6]:
projected_members_graph = H

In [7]:
update_networkx_nodes(projected_members_graph, updated_projected_members_nodelist)


In [57]:
nx.write_graphml(projected_members_graph, 'borrow_events_unipartite_projected_members.graphml')

In [8]:

nodelist = pd.merge(projected_members[['uri', 'borrow_count']], updated_projected_members_nodelist, on='uri')

In [11]:
generate_corr_chart(nodelist, 'test')


alt.LayerChart(...)

In [48]:
nodelist.groupby('global_layer').size()

global_layer
0     94
1    100
2    160
3    244
dtype: int64

In [49]:
nodelist.columns

Index(['uri', 'borrow_count', 'node', 'global_hub_spoke', 'global_layer',
       'global_coreness', '_nx_name', 'group', 'bipartite', 'global_degree',
       'local_degree', 'global_eigenvector', 'local_eigenvector',
       'global_closeness', 'local_closeness', 'global_betweenness',
       'local_betweenness', 'global_clustering', 'local_clustering',
       'node_title', 'global_graph_radius', 'global_diameter',
       'local_graph_radius', 'local_diameter', 'component', 'local_katz',
       'local_louvain', 'local_pagerank', 'local_hubs', 'local_auth',
       'pagerank', 'hubs', 'auth', 'global_pagerank', 'global_hubs',
       'global_auth', 'global_katz', 'global_louvain', 'label'],
      dtype='object')

In [51]:
nodelist.local_louvain.value_counts()

0.0    186
1.0    171
2.0    144
3.0     81
Name: local_louvain, dtype: int64

In [52]:
nodelist.global_louvain.value_counts()


0.0    177
1.0    167
2.0    143
3.0     94
4.0      2
Name: global_louvain, dtype: int64

In [55]:
nodelist[nodelist.local_louvain == 0].global_louvain.value_counts()

0.0    60
1.0    52
2.0    46
3.0    22
4.0     1
Name: global_louvain, dtype: int64